# Multiple Inheritance 
In python is **not** much more complex than single inheritance.





Remeber to use **isinstance()** and **issubclass()**


In [1]:
help(isinstance )

Help on built-in function isinstance in module builtins:

isinstance(obj, class_or_tuple, /)
    Return whether an object is an instance of a class or of a subclass thereof.
    
    A tuple, as in ``isinstance(x, (A, B, ...))``, may be given as the target to
    check against. This is equivalent to ``isinstance(x, A) or isinstance(x, B)
    or ...`` etc.



In [2]:
help(issubclass)

Help on built-in function issubclass in module builtins:

issubclass(cls, class_or_tuple, /)
    Return whether 'cls' is a derived from another class or is the same class.
    
    A tuple, as in ``issubclass(x, (A, B, ...))``, may be given as the target to
    check against. This is equivalent to ``issubclass(x, A) or issubclass(x, B)
    or ...`` etc.



In [4]:
# Test it 
isinstance(3, int)

True

In [5]:
isinstance("Hello", str)

True

In [6]:
isinstance(4.1258, bytes)

False

In [7]:
# isinstance accepts tuple of types 
# for the second argument 
x = []
isinstance(x,(float, dict,list))

True

In [42]:
class SimpleList():
    """
    Sorted list example
    """
    
    def __init__(self, items):
        self._items = items

    def add(self, item):
        self._items.append(item)
        
    def __getitem__(self, index):
        return self._items[index]
    
    def sort(self):
        self._items.sort()
        
    def __len__(self):
        return len(self._items)
    
    def __repr__(self):
        return "SimpleList({!r})".format(self._items)

In [56]:
class SortedList(SimpleList):
    
    def __init__(self, items=()):
        super().__init__(items)
        self.sort()
    
    #overwrite Method
    def add(self, item):
        super().add(item)
        self.sort()
        
    def __repr__(self):
        return "SortedList({!r})".format(list(self))

In [57]:
# Test it
sl = SortedList([4, 5, 2, 99, 11])
sl

SortedList([2, 4, 5, 11, 99])

In [58]:
len(sl) #looks for __len__

5

In [59]:
sl.add(-6)
sl

SortedList([-6, 2, 4, 5, 11, 99])

In [60]:
class IntList(SimpleList):
    #call init on every item in the list 
    def __init__(self, items=()):
        for x in items:
            self._validate(x)
        super().__init__(items)
        
    @staticmethod
    def _validate(x):
        if not isinstance(x, int):
            raise TypeError("IntList only supports integer values")
            
    def add(self, item):
        """
        Validate the item before you add it to the list 
        """
        self._validate(item)
        super().add(item)
        
    def __repr__(self):
        return "IntList({!r})".format(list(self))

In [61]:
# Test it 
il = IntList([1, 2, 8, 9])
il.add(29)
il

IntList([1, 2, 8, 9, 29])

### Multiple inheritance 
Define a class with more than one base class <br>
This is not a universal functionality of OO languages.

For exapmle C++ Does support it, and Java does not.

Note: It could lead to some complicated situations. 

- What if more than one base class defines a particular method?

Python has a relative simple and understandable system for **multiple inheritance**

syntax: **class SubClass(Base1, Base2, ...)**

- Subclasses inherit methods from All bases
- Without conflict, names resolve in the obvious way.
- **Method Resolution Order (MRO)** Determines name lookup in all cases

In [62]:
class SortedIntList(IntList, SortedList):
    def __repr__(self):
        return "SortedIntList({!r})".format(list(self))

In [63]:
# Test it 
sil = SortedIntList([42, 2, 17, 9])
sil

SortedIntList([2, 9, 17, 42])

In [64]:
# Test it 
sil = SortedIntList([42, 2, 17, "NaN"])
sil

TypeError: IntList only supports integer values

In [65]:
# Add method maintains both the sorting and type constraints 
sil.add(-12)
sil

SortedIntList([-12, 2, 9, 17, 42])

**How does Python know which add() to call?** <br>
**How does Python maintain both constraints?**<br>

The answer to both of these questions is, the **resoultion order**

**MRO and Super()** how they work.

If a class
- has **multiple** base classes
- defines **no initializer** then **only** the initializer of the **first** base class is automatically called.

In [68]:
class Base1:
    def __init__(self):
        print("Base1.__init__")
        
class Base2:
    def __init__(self):
        print("Base2.__init__")
        
class Sub(Base1, Base2):
        pass
        


In [69]:
# Test it
s = Sub()

Base1.__init__


Through the use of **super()** we could design these classes such that both Base1 and Base2 are calledautomatically.  


**\_\_bases\_\_**: A tuple of base classes

In [71]:
SortedIntList.__bases__

(__main__.IntList, __main__.SortedList)

In [72]:
IntList.__bases__

(__main__.SimpleList,)

## Method Resolution Order (MRO)
ordering that determines method name lookup 
- Methods may be defined in multiple places 
- MRO is an ordering of the inheritance graph 
- It is quite simple!

Let's Look at an example,  but let's look at where our MRO is stored.

**\_\_mro\_\_**


In [73]:
SortedIntList.__mro__

(__main__.SortedIntList,
 __main__.IntList,
 __main__.SortedList,
 __main__.SimpleList,
 object)

In [74]:
# Get it in list form
SortedIntList.mro()

[__main__.SortedIntList,
 __main__.IntList,
 __main__.SortedList,
 __main__.SimpleList,
 object]

## How is MRO used?
**obj.method()**
class SomeClass
1. instance of:
    - Base1!
    - Base2!
    - Base3! Yes. a hit here 
    - Base4!
    
2. MRO
    - match!
    - Base3.method(obj)

3. Resolves to
    - ex: 19

In [75]:
class A:
    def func(self):
        return "A.func"

class B(A):
    def func(self):
        return "B.func"
    
class C(A):
    def func(self):
        return "C.func"
    
class D(C, B):
    pass

In [76]:
D.mro()

[__main__.D, __main__.C, __main__.B, __main__.A, object]

In [79]:
# What should you expect when typing d.fun()
d = D()
d.func()

'C.func'

In [80]:
# Change the order of base classes 
class D(B, C):
    pass

In [82]:
d = D()
d.func()

'B.func'

In [83]:
SortedIntList.mro()


[__main__.SortedIntList,
 __main__.IntList,
 __main__.SortedList,
 __main__.SimpleList,
 object]

This is properly maintaining both, their sorting constraints and type constraints of both SortedList



**How is IntList.add() deferring to SortedList.add()?**

The answer is how **super()** actually works.


### How does Python calculate the MRO?
C3: Algorithm for calculating MRO in Python
- Subclasses come **before** base classes
- Base Class order from class definition is **preserved**.
- First two qualities are preserved **no matter** where you start in the inheritance graph.


Note: Not all inheritance declarations are allowed 

In [84]:
class A:
    pass

class B(A):
    pass
    
class C(A):
    pass
    
class D(B, A, C):
    pass

TypeError: Cannot create a consistent method resolution
order (MRO) for bases C, A

Note:  Since both, B and C inherit from A, they **must** come before A.  


C3 cannot put A both before and after C.

## The built-in super() function
TODO:(come back to it)
